# Some feature engineering (part I)

Kaggle competitions are won by **feature engineering**: those who win are those who can create the most useful features out of the data. This is true for the most part as the winning models, at least for structured data, all tend to be variants of [gradient boosting](http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/). 

This represents one of the patterns in machine learning: feature engineering has a **greater return on investment** than model building and hyperparameter tuning. As Andrew Ng is fond of saying: "applied machine learning is basically feature engineering."

While choosing the right model and optimal settings are important, the **model can only learn from the data it is given**. Making sure this data is as relevant to the task as possible is the job of the data scientist (and maybe some [automated tools](https://docs.featuretools.com/getting_started/install.html) to help us out).

Feature engineering refers to a general process and can involve both **feature construction**, i.e., _adding new features from the existing data_, and **feature selection**, i.e., _choosing only the most important features or other methods of dimensionality reduction_. There are many techniques we can use to both create features and select features.

We'll start by trying out two simple feature construction methods:

- Domain knowledge features
- Polynomial features

## Imports

In [ ]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## Let's deifne some auxiliary function definitions we'll use later on...

In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_encode(app_train, app_test) : 
    le = LabelEncoder()
    le_count = 0

    # Iterate through the columns
    for col in app_train:
        if app_train[col].dtype == 'object':
            # If 2 or fewer unique categories
            set_values = app_train[col].unique()
            num_values = len(list(set_values))
            if num_values <= 2:
                print(f"{col} will be label encoded! Found {num_values} values: {set_values}")
                # Train on the training data
                le.fit(app_train[col])
                # Transform both training and testing data
                app_train[col] = le.transform(app_train[col])
                app_test[col] = le.transform(app_test[col])

                # Keep track of how many columns were label encoded
                le_count += 1

    print('%d columns were label encoded.' % le_count)
    print('Training Features shape: ', app_train.shape)
    print('Testing Features shape: ', app_test.shape)
    
    return app_train, app_test


def one_hot_encode(app_train, app_test) :
    
    # Let's perform the one-hot encoding of categorical features with > 2 values...
    app_train = pd.get_dummies(app_train)
    app_test = pd.get_dummies(app_test)
    print('Training Features shape: ', app_train.shape)
    print('Testing Features shape: ', app_test.shape)
    
    return app_train, app_test


def align_train_test(app_train, app_test) :
    
    # Save target variable in a separate Series...
    train_labels = app_train['TARGET']

    # Align the training and testing data on columns -- this keeps only the columns present in both dataframes.
    app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

    # Add the target column back in.
    app_train['TARGET'] = train_labels
    
    return train_labels, app_train, app_test

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm.sklearn import LGBMClassifier

def classify(app_train, app_test, classifier = 'logistic', cv = 5, param_grid = {}, n_jobs = 1) :
    
    # Pick the classifier desired by the user...
    log_reg = LogisticRegression
    if classifier == 'decision' : log_reg = DecisionTreeClassifier
    elif classifier == 'SVC' : log_reg = SVC
    elif classifier == 'rf' : log_reg = RandomForestClassifier
    elif classifier == 'lgbm' : log_reg = LGBMClassifier
    else : pass
    
    
    # Label encoding, one hot encoding, train-test alignment.
    app_train, app_test = label_encode(app_train, app_test)
    app_train, app_test = one_hot_encode(app_train, app_test)
    train_labels, app_train, app_test = align_train_test(app_train, app_test)
    
    
    train = app_train.drop(columns = ['TARGET'], errors = 'ignore')
    test = app_test.copy()
    
 
    # Setup the pipeline. 
    from sklearn.pipeline import Pipeline
    clf = Pipeline([('imp', SimpleImputer(strategy = 'median')),
                    ('sca', MinMaxScaler(feature_range = (0, 1))),
                    ('clf', log_reg(class_weight = 'balanced'))],
                    verbose = True)
    

    # Setup the grid search. 
    from sklearn.model_selection import GridSearchCV
    search = GridSearchCV(estimator = clf, 
                          param_grid = param_grid, 
                          cv = cv,
                          scoring = 'roc_auc',
                          n_jobs = n_jobs,
                          verbose = 1)
    
    
    # Training ...
    search.fit(train, train_labels)

    
    #print('Miscellanea of results:', search.cv_results_)
    #print('Score achieved by the best config. during stratified CV:', search.best_score_)
    #print('Best estimator config:', search.best_estimator_)

    
    # Compute predictions...
    log_reg_pred = search.predict_proba(test)[:, 1]
    # print(log_reg_pred)

    
    # Final result dataframe.
    submit = app_test[['SK_ID_CURR']]
    submit['TARGET'] = log_reg_pred
    #submit.head()

    
    # Compute feature importance (if applicable).
    feat_imp = pd.Series(0, index=train.columns)
    if(classifier in ['rf', 'lgbm']) :
        feat_imp = pd.Series(search.best_estimator_.named_steps["clf"].feature_importances_, index=train.columns)
        
    # Return the results.
    return submit, search.cv_results_, feat_imp

## Read in Data 

First, we can list all the available data files. There are a total of 9 files: 1 main file for training (with target) 1 main file for testing (without the target), 1 example submission file, and 6 other files containing additional information about each loan. 

In [ ]:
# Training data
app_train = pd.read_csv('./input/application_train.csv')
app_test = pd.read_csv('./input/application_test.csv')

## Domain Knowledge Features

Maybe it's not entirely correct to call this "domain knowledge" because I'm not a credit expert, but perhaps we could call this "attempts at applying limited financial knowledge". In this frame of mind, we can make a couple features that attempt to capture what we think may be important to predict whether a client will default on a loan. 

Here we're going to use four features that were inspired by [this script](https://www.kaggle.com/jsaguiar/updated-0-792-lb-lightgbm-with-simple-features) by Aguiar:

* `CREDIT_INCOME_PERCENT`: the percentage of the credit amount relative to a client's income
* `ANNUITY_INCOME_PERCENT`: the percentage of the loan annuity relative to a client's income
* `CREDIT_TERM`:  the length of the payment in months (since the annuity is the monthly amount due)
* `DAYS_EMPLOYED_PERCENT`: the percentage of the days employed relative to the client's age

In [ ]:
# Let's copy the dataframes into two new variables...
app_train_domain = app_train.copy()
app_test_domain = app_test.copy()

Let's then generate the four features mentioned above. Notice how we create them both in the training set AND in the test set.

In [ ]:
app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']

In [ ]:
app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']

#### Visualize New Variables

We should explore these __domain knowledge__ variables visually in a graph. For all of these, we will make the same KDE plot colored by the value of the `TARGET`.

In [ ]:
plt.figure(figsize = (12, 20))
# iterate through the new features
for i, feature in enumerate(['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT']):
    
    # create a new subplot for each source
    plt.subplot(4, 1, i + 1)
    # plot repaid loans
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 0, feature], label = 'target == 0')
    # plot loans that were not repaid
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 1, feature], label = 'target == 1')
    
    # Label the plots
    plt.title('Distribution of %s by Target Value' % feature)
    plt.xlabel('%s' % feature); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)

It's hard to say ahead of time if these new features will be useful. The only way to tell for sure is to try them out! 

In [ ]:
param_grid_log = {'clf__C': [0.0001, 0.0005]}
param_grid_tree = {'clf__max_depth': [5, 7, 10]}
param_grid_rf = {'clf__n_estimators': [150], 'clf__max_depth': [7, 10], 'clf__n_jobs' : [-1]}
param_grid_lgbm = {'clf__objective': ['binary'],
                   'clf__metric': ['auc'],
                   'clf__n_estimators': [150], 
                   'clf__max_depth': [7, 10]}

predictions, search_res, feat_imp_domain = classify(app_train_domain.copy(),
                                      app_test_domain.copy(),
                                      classifier = 'rf', 
                                      cv = 3, 
                                      param_grid = param_grid_rf,
                                      n_jobs = 1)

for par, score in zip(search_res['params'], search_res['mean_test_score']) :
    print(f"Config: {par} -- score: {score}")

Let's now train our classifier **without the new features**, and compare the results...

In [ ]:
param_grid_log = {'clf__C': [0.0001, 0.0005]}
param_grid_tree = {'clf__max_depth': [5, 7, 10]}
param_grid_rf = {'clf__n_estimators': [150], 'clf__max_depth': [7, 10], 'clf__n_jobs' : [-1]}
param_grid_lgbm = {'clf__objective': ['binary'],
                   'clf__metric': ['auc'],
                   'clf__n_estimators': [150], 
                   'clf__max_depth': [7, 10]}

predictions_base, search_res_base, feat_imp_base = classify(app_train.copy(),
                                                 app_test.copy(),
                                                 classifier = 'rf',
                                                 cv = 3, 
                                                 param_grid = param_grid_rf,
                                                 n_jobs = 1)

for par, score in zip(search_res_base['params'], search_res_base['mean_test_score']) :
    print(f"Config: {par} -- score: {score}")

In [ ]:
predictions_base.to_csv('base.csv', index = False)
predictions.to_csv('add_feats.csv', index = False)

## Polynomial Features

One simple feature construction method is called [polynomial features](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html). In this method, we make features that are **powers** of existing features as well as **interaction terms** between existing features. For example, we can:

- create variables `EXT_SOURCE_1^2` and `EXT_SOURCE_2^2` ...
- ... also variables such as `EXT_SOURCE_1` x `EXT_SOURCE_2`, `EXT_SOURCE_1` x `EXT_SOURCE_2^2`, `EXT_SOURCE_1^2` x   `EXT_SOURCE_2^2`, and so on. 

These features that are a combination of multiple individual variables are called [interaction terms](https://en.wikipedia.org/wiki/Interaction_(statistics) because they capture the interactions between variables. In other words, while two variables by themselves may not have a strong influence on the target, combining them together into a single interaction variable might show a relationship with the target. [Interaction terms are commonly used in statistical models](https://www.theanalysisfactor.com/interpreting-interactions-in-regression/) to capture the effects of multiple variables, but I do not see them used as often in machine learning. Nonetheless, we can try out a few to see if they might help our model to predict whether or not a client will repay a loan. 

Jake VanderPlas writes about [polynomial features in his excellent book Python for Data Science](https://jakevdp.github.io/PythonDataScienceHandbook/05.04-feature-engineering.html) for those who want more information.

In the following code, we create polynomial features using the `EXT_SOURCE` variables and the `DAYS_BIRTH` variable. [Scikit-Learn has a useful class called `PolynomialFeatures`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) that creates the polynomials and the interaction terms up to a specified degree. We can use a degree of 3 to see the results (when we are creating polynomial features, we want to avoid using too high of a degree, both because the number of features scales exponentially with the degree, and because we can run into [problems with overfitting](http://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html#sphx-glr-auto-examples-model-selection-plot-underfitting-overfitting-py)). 

In [ ]:
# Training data
app_train = pd.read_csv('./input/application_train.csv')
app_test = pd.read_csv('./input/application_test.csv')

In [ ]:
# Make a new dataframe for polynomial features
poly_features = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'TARGET']]
poly_features_test = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]

# imputer for handling missing values
imputer = SimpleImputer(strategy = 'median')

poly_target = poly_features['TARGET']
poly_features = poly_features.drop(columns = ['TARGET'])

# Need to impute missing values
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.transform(poly_features_test)

from sklearn.preprocessing import PolynomialFeatures
                                  
# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 3)

In [ ]:
# Train the polynomial features
poly_transformer.fit(poly_features)

# Transform the features
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

This creates a **considerable number of new features**. To get the names we have to use the polynomial features `get_feature_names` method.

In [ ]:
poly_transformer.get_feature_names(input_features = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])

There are 35 features with individual features raised to powers up to degree 3 and interaction terms. Now, let's see whether any of these new features **are correlated with the target**.

In [ ]:
# Put train features into dataframe
poly_features = pd.DataFrame(poly_features, 
                             columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))
print(poly_features.shape)
print(poly_features.columns.values)

# Add in the target
poly_features['TARGET'] = poly_target

# Find the correlations with the target
poly_corrs = poly_features.corr()['TARGET'].sort_values()

# Display most negative and most positive
print(poly_corrs.head(10))
print(poly_corrs.tail(5))

Several of the new variables have a greater (in terms of absolute magnitude) correlation with the target than the original features. When we build machine learning models, we can try with and without these features to determine if they actually help the model learn. 

We will add these features to a copy of the training and testing data and then evaluate models with and without the features. Many times in machine learning, the only way to know if an approach will work is to try it out! 

In [ ]:
# Put test features into dataframe
poly_features_test = pd.DataFrame(poly_features_test, 
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                                'EXT_SOURCE_3', 'DAYS_BIRTH']))

# Merge polynomial features into training dataframe
poly_features['SK_ID_CURR'] = app_train['SK_ID_CURR']
app_train_poly = app_train.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

# Merge polnomial features into testing dataframe
poly_features_test['SK_ID_CURR'] = app_test['SK_ID_CURR']
app_test_poly = app_test.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')

# Align the dataframes
app_train_poly, app_test_poly = app_train_poly.align(app_test_poly, join = 'inner', axis = 1)

app_train_poly['TARGET'] = poly_target

# Print out the new shapes
print('Training data with polynomial features shape: ', app_train_poly.shape)
print('Testing data with polynomial features shape:  ', app_test_poly.shape)

It's hard to say ahead of time if these new features will be useful. The only way to tell for sure is to try them out! 

### Make Predictions using Engineered Features

The only way to see if the Polynomial Features and Domain knowledge improved the model is to train a test a model on these features! We can then compare the submission performance to that for the model without these features to gauge the effect of our feature engineering.

In [ ]:
param_grid_log = {'clf__C': [0.0001, 0.0005]}
param_grid_tree = {'clf__max_depth': [5, 7, 10]}
param_grid_rf = {'clf__n_estimators': [150], 'clf__max_depth': [7, 10], 'clf__n_jobs' : [-1]}
param_grid_lgbm = {'clf__objective': ['binary'],
                   'clf__metric': ['auc'],
                   'clf__n_estimators': [150], 
                   'clf__max_depth': [7, 10]}

predictions_poly, search_res_poly, feat_imp_poly = classify(app_train_poly.copy(),
                                            app_test_poly.copy(),
                                            classifier = 'rf',
                                            cv = 3, 
                                            param_grid = param_grid_rf,
                                            n_jobs = 1)

for par, score in zip(search_res_poly['params'], search_res_poly['mean_test_score']) :
    print(f"Config: {par} -- score: {score}")

In [ ]:
predictions_poly.to_csv('poly.csv', index = False)
predictions_poly.shape

Depending on the model and the hyperparameters used, polynomial features may **improve classification accuracy**.

## Model Interpretation: Feature Importances

As a simple method to see which variables are the most relevant, we can look at the feature importances of the random forest. Given the correlations we saw in the exploratory data analysis, we should expect that the most important features are the `EXT_SOURCE` and the `DAYS_BIRTH`. We may use these feature importances as a method of dimensionality reduction in future work.

In [ ]:
feat_imp_poly.sort_values(ascending = False).head(50)

In [ ]:
def plot_feature_importances(df):
    """
    Plot importances returned by a model. This can work with any measure of
    feature importance provided that higher importance is better. 
    
    Args:
        df (dataframe): feature importances. Must have the features in a column
        called `features` and the importances in a column called `importance
        
    Returns:
        shows a plot of the 15 most importance features
        
        df (dataframe): feature importances sorted by importance (highest to lowest) 
        with a column for normalized importance
        """
    
    # Sort features according to importance
    df = df.sort_values(ascending = False)
    
    # Normalize the feature importances to add up to one
    df_norm = df / df.sum()

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()
    
    # Need to reverse the index to plot most important on top
    ax.barh(list(df_norm.index[:15]), 
            df_norm.head(15), 
            align = 'center', edgecolor = 'k')
    
    # Set the yticks and labels
    ax.set_yticks(list(df_norm.index[:15]))
    
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()
    
    return df

In [ ]:
# Show the feature importances for the default features
feature_importances_sorted = plot_feature_importances(feat_imp_poly)

As expected, the most important features are those dealing with `EXT_SOURCE` and `DAYS_BIRTH`. We see that there are only a handful of features with a significant importance to the model, which suggests we may be able to drop many of the features without a decrease in performance (and we may even see an increase in performance.) Feature importances are not the most sophisticated method to interpret a model or perform dimensionality reduction, but they let us start to understand what factors our model takes into account when it makes predictions. 

In [ ]:
feature_importances_domain_sorted = plot_feature_importances(feat_imp_domain)

We see that all four of our hand-engineered features made it into the top 15 most important! This should give us confidence that our domain knowledge was at least partially on track.